In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np, os
import matplotlib.pyplot as plt

from pathlib import Path
# import yaml
# import re
# import datetime

# import pyarrow as pa
# import pyarrow.parquet as pq

# import gbd_mapping as gm
# from vivarium import Artifact

# from db_queries import get_ids, get_outputs, get_population, get_covariate_estimates
# from get_draws.api import get_draws

import vivarium_helpers as vh
# import vivarium_helpers.id_helper as idh
# from vivarium_helpers.vph_output.operations import VPHOperator
# from vivarium_helpers.vph_output.measures import VPHResults
from vivarium_helpers.utils import convert_to_categorical, constant_categorical, print_memory_usage, Timer
# from vivarium_helpers.vph_output import cleaning
# from vivarium_helpers.vph_output.loading import load_draws_from_keyspace_files
from vivarium_helpers.projects.alzheimers import loading, population
from vivarium_helpers.projects.alzheimers.population import RunType
# from vivarium_helpers.vph_artifact.operations import convert_to_sim_format
from vivarium_helpers.projects.alzheimers.results import AlzheimersResultsProcessor

!date
!whoami
!pwd

Thu Nov 20 23:03:38 PST 2025
ndbs
/mnt/share/code/ndbs/vivarium_research_alzheimers/results_tables


In [3]:
# from vivarium_helpers.projects.alzheimers.results import AlzheimersResultsProcessor

# Define data directories

In [4]:
# Project directory
project_dir = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/')

# Output directory to store final results for client
output_dir = Path(r"J:\Project\simulation_science\alzheimers\results_in_progress".replace('\\', '/').replace('J:', '/snfs1'))
print(output_dir.exists())

# Output from multistate life table
mslt_output_dir = output_dir.parent / 'results_11_14_2025_mslt'
print(mslt_output_dir.exists())
output_dir

True
True


PosixPath('/snfs1/Project/simulation_science/alzheimers/results_in_progress')

In [5]:
# # For testing: Run directory containing model 8.3 results for all
# # locations
# model_run_subdir = 'results/abie_consistent_model_test/united_states_of_america/2025_10_28_08_55_05/'

# # Results directory for model 8.3, for testing
# results_dirs = project_dir / model_run_subdir / 'results/'

# Artifact for models 8.3 - 8.7
artifact_model_number = '8.3'


In [6]:
# Model 8.4 results (final runs for 10/31/25 intermediate results)
# Each batch run contains all locations, 100 random seeds, and 3 or 4 draws
model_run_dir_8_4 = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.4/model_spec')

# Model 8.7 results (updated final run completed on 11/10/2025)
model_run_dir_8_7 = Path('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec')

# Batch runs are stored in "timestamp" subdirectories of the model run
# directory
with os.scandir(model_run_dir_8_7) as entries:
    batch_run_dirs_8_7 = [entry.path for entry in entries]

# Select a model and filter for testing
batch_run_dirs = batch_run_dirs_8_7[:2]

# batch_results_dirs = []
# for run_dir in batch_run_dirs:
#     if run_dir.endswith('2025_10_29_20_41_39'):
#         # One batch had to be deduplicated
#         results_dir = run_dir + '/deduplicated_results'
#     else:
#         results_dir = run_dir + '/results'
#     batch_results_dirs.append(results_dir)
# batch_results_dirs

# batch_results_dirs
batch_run_dirs

['/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29',
 '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_06_06_42_37']

# Create an AlzheimersResultsProcessor object to process results

In [7]:
results = AlzheimersResultsProcessor('8.3', RunType.FINAL, batch_run_dirs=batch_run_dirs)

In [8]:
results.initial_simulation_population

2000000

In [9]:
results.locations

['United States of America',
 'Brazil',
 'China',
 'Germany',
 'Israel',
 'Japan',
 'Spain',
 'Sweden',
 'Taiwan (Province of China)',
 'United Kingdom']

In [10]:
results.draws

[457, 169, 323, 258, 446]

# Load population data and calculate model scale

In [11]:
with Timer():
    results.load_population_data()
results.ops.describe(results.model_scale)

Elapsed time: 0:00:16.256168


,count,mean,std,min,25%,50%,75%,max
location,,,,,,,,
Brazil,5.0,1.038135,0.079817,0.943491,0.982799,1.053388,1.059915,1.151080
China,5.0,0.116134,0.007398,0.106308,0.112346,0.116177,0.120095,0.125744
Germany,5.0,0.924287,0.055826,0.870607,0.885982,0.912870,0.939648,1.012328
Israel,5.0,42.079082,2.676754,39.273048,39.388529,42.591991,43.913069,45.228774
Japan,5.0,0.577381,0.072172,0.511002,0.534305,0.554160,0.592567,0.694869
Spain,5.0,3.173615,0.111936,3.000791,3.175186,3.187141,3.190730,3.314228
Sweden,5.0,12.967456,1.269209,11.361366,12.447040,12.890094,13.308238,14.830542
Taiwan (Province of China),5.0,8.645158,0.498290,8.050507,8.202807,8.855923,8.887794,9.228760
United Kingdom,5.0,2.012453,0.094787,1.905066,1.969920,1.987250,2.042450,2.157581


In [12]:
results.person_time

,location,sex,event_year,input_draw,value,age_group
0,United States of America,Female,2025,169,1.156020e+07,25_to_29
1,United States of America,Female,2025,258,1.094300e+07,25_to_29
2,United States of America,Female,2025,323,1.051324e+07,25_to_29
3,United States of America,Female,2025,446,1.097049e+07,25_to_29
4,United States of America,Female,2025,457,1.121058e+07,25_to_29
...,...,...,...,...,...,...
113995,United Kingdom,Male,2100,169,1.309761e+05,95_plus
113996,United Kingdom,Male,2100,258,1.554175e+05,95_plus
113997,United Kingdom,Male,2100,323,1.205289e+05,95_plus
113998,United Kingdom,Male,2100,446,1.254920e+05,95_plus


# Load and process deaths data for testing

In [13]:
# with Timer():
#     results.load_measure_from_batch_runs('deaths', filters=loading.FINAL_RESULTS_FILTERS['deaths'])

In [14]:
# results.data.deaths

In [15]:
# deaths_prepped = results.process_deaths(results.data.deaths)
# deaths_prepped

# Preprocess MSLT results

In [16]:
!ls $mslt_output_dir

2025_11_14__14_11_42_outside_sim_results_draw_level.csv


In [17]:
mslt_results = pd.read_csv(mslt_output_dir / '2025_11_14__14_11_42_outside_sim_results_draw_level.csv', index_col=0)
mslt_results

,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Draw,Value
0,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,113,0.000000
1,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,13,0.000000
2,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,158,0.000000
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,169,0.000000
4,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,177,0.000000
...,...,...,...,...,...,...,...,...,...,...
151995,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,46,41101.303607
151996,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,460,41691.942867
151997,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,480,39195.330257
151998,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,499,40424.816911


In [18]:
mslt_results.Age.unique()

array(['60_to_64', '65_to_69', '70_to_74', '75_to_79'], dtype=object)

In [19]:
mslt_results_prepped = results.process_mslt_results(mslt_results)
mslt_results_prepped

,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,169,0.000000
8,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,258,0.000000
13,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,323,0.000000
18,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,446,0.000000
19,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,457,0.000000
...,...,...,...,...,...,...,...,...,...,...
151978,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,169,42594.508978
151983,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,258,40580.387735
151988,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,323,38815.152311
151993,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,446,40402.097304


# BBBM Test Counts

In [20]:
filename = 'counts_bbbm_tests'
# short_name = 'bbbm_tests'
with Timer():
    bbbm_tests = results.load_measure_from_batch_runs(
        filename, False, filters=loading.FINAL_RESULTS_FILTERS[filename])

bbbm_tests

Timer BatchLoadingTimer started
[('event_year', '>=', '2025'), ('bbbm_test_results', '!=', 'not_tested'), ('age_group', 'in', ['60_to_64', '65_to_69', '70_to_74', '75_to_79'])]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spa

,age_group,artifact_path,bbbm_test_results,entity,entity_type,event_year,input_draw,measure,scenario,sex,value,location
0,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,baseline,Female,0.0,Japan
1,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,baseline,Male,0.0,Japan
2,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,bbbm_testing,Female,0.0,Japan
3,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,bbbm_testing,Male,0.0,Japan
4,60_to_64,/mnt/team/simulation_science/pub/models/vivari...,positive,bbbm_testing,testing,2025,169,counts_bbbm_tests,bbbm_testing_and_treatment,Female,0.0,Japan
...,...,...,...,...,...,...,...,...,...,...,...,...
182395,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,negative,bbbm_testing,testing,2100,446,counts_bbbm_tests,baseline,Male,0.0,Brazil
182396,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,negative,bbbm_testing,testing,2100,446,counts_bbbm_tests,bbbm_testing,Female,2445.0,Brazil
182397,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,negative,bbbm_testing,testing,2100,446,counts_bbbm_tests,bbbm_testing,Male,1218.0,Brazil
182398,75_to_79,/mnt/team/simulation_science/pub/models/vivari...,negative,bbbm_testing,testing,2100,446,counts_bbbm_tests,bbbm_testing_and_treatment,Female,2445.0,Brazil


In [21]:
bbbm_tests.age_group.unique()

['60_to_64', '65_to_69', '70_to_74', '75_to_79']
Categories (15, object): ['25_to_29' < '30_to_34' < '35_to_39' < '40_to_44' ... '80_to_84' < '85_to_89' < '90_to_94' < '95_plus']

In [22]:
bbbm_tests.query("value>0").age_group.unique()

['60_to_64', '65_to_69', '70_to_74', '75_to_79']
Categories (15, object): ['25_to_29' < '30_to_34' < '35_to_39' < '40_to_44' ... '80_to_84' < '85_to_89' < '90_to_94' < '95_plus']

In [23]:
bbbm_tests_prepped = results.process_bbbm_tests(bbbm_tests, mslt_results_prepped)
bbbm_tests_prepped

,age_group,event_year,input_draw,location,measure,scenario,sex,value,disease_stage,metric
0,60_to_64,2025,169,Japan,BBBM Tests,baseline,Female,0.000000,Preclinical AD,Number
1,60_to_64,2025,169,Japan,BBBM Tests,baseline,Male,0.000000,Preclinical AD,Number
2,60_to_64,2025,169,Japan,BBBM Tests,bbbm_testing,Female,0.000000,Preclinical AD,Number
3,60_to_64,2025,169,Japan,BBBM Tests,bbbm_testing,Male,0.000000,Preclinical AD,Number
4,60_to_64,2025,169,Japan,BBBM Tests,bbbm_testing_and_treatment,Female,0.000000,Preclinical AD,Number
...,...,...,...,...,...,...,...,...,...,...
303995,75_to_79,2100,169,United States of America,BBBM Positive Tests,BBBM Testing and Treatment,Male,141981.696594,Susceptible,Number
303996,75_to_79,2100,258,United States of America,BBBM Positive Tests,BBBM Testing and Treatment,Male,135267.959115,Susceptible,Number
303997,75_to_79,2100,323,United States of America,BBBM Positive Tests,BBBM Testing and Treatment,Male,129383.841038,Susceptible,Number
303998,75_to_79,2100,446,United States of America,BBBM Positive Tests,BBBM Testing and Treatment,Male,134673.657679,Susceptible,Number


In [24]:
with Timer():
    bbbm_tests_output = results.summarize_and_beautify(bbbm_tests_prepped, 'disease_stage')
bbbm_tests_output

Timer SummarizingTimer started
2025-11-20 23:04:09.279160
2025-11-20 23:04:10.768355
2025-11-20 23:06:35.000863
Elapsed time: 145.7290 seconds
Elapsed time: 0:02:25.729306


,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,60_to_64,Female,Preclinical AD,Reference,BBBM Tests,Number,0.000000,0.000000,0.000000
1,2025,Brazil,60_to_64,Male,Preclinical AD,Reference,BBBM Tests,Number,0.000000,0.000000,0.000000
2,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing Only,BBBM Tests,Number,0.000000,0.000000,0.000000
3,2025,Brazil,60_to_64,Male,Preclinical AD,BBBM Testing Only,BBBM Tests,Number,0.000000,0.000000,0.000000
4,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing and Treatment,BBBM Tests,Number,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
121595,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,BBBM Positive Tests,Number,310330.641935,281236.426008,341065.408694
121596,2100,United States of America,75_to_79,Female,Susceptible,BBBM Testing Only,BBBM Positive Tests,Rate,0.043456,0.040911,0.045766
121597,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,BBBM Positive Tests,Rate,0.047640,0.044402,0.050626
121598,2100,United States of America,75_to_79,Female,Susceptible,BBBM Testing and Treatment,BBBM Positive Tests,Rate,0.043456,0.040911,0.045766


In [25]:
# Save BBBM testing results
bbbm_tests_output.to_csv(output_dir / 'bbbm_tests.csv', index=False)

# CSF and PET Test Counts

In [26]:
csf_pet_filters = [('testing_state', 'in', ['csf', 'pet'])]
# filename = 'counts_baseline_tests_among_eligible'
with Timer():
    csf_pet_tests = results.load_measure_from_batch_runs(
        'counts_baseline_tests_among_eligible', False, filters=csf_pet_filters
    )
print(len(csf_pet_tests), 'rows')
csf_pet_tests.head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025'), ('testing_state', 'in', ['csf', 'pet'])]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/japan.hdf', 'Spain': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/art

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,testing_state,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Female,csf,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Female,pet,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Male,csf,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,baseline,Male,pet,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,counts_baseline_tests_among_eligible,bbbm_testing,Female,csf,0.0,Japan


In [27]:
csf_pet_tests.query("value>0").age_group.unique()

['30_to_34', '35_to_39', '40_to_44', '45_to_49', '50_to_54', ..., '75_to_79', '80_to_84', '85_to_89', '90_to_94', '95_plus']
Length: 14
Categories (15, object): ['25_to_29' < '30_to_34' < '35_to_39' < '40_to_44' ... '80_to_84' < '85_to_89' < '90_to_94' < '95_plus']

In [28]:
csf_pet_tests_prepped = results.process_csf_pet_tests(csf_pet_tests)
csf_pet_tests_prepped

19.842122 MB measure
6.618122 MB minuend
13.230122 MB subtrahend
4.341673 MB minuend re-indexed
8.673673 MB subtrahend re-indexed
8.673733 MB difference
15.054254 MB difference with reset index
15.510579 MB final difference


,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,testing_state,value,location,metric,disease_stage
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,CSF Tests,baseline,Female,csf,0.0,Japan,Number,MCI due to AD
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,PET Tests,baseline,Female,pet,0.0,Japan,Number,MCI due to AD
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,CSF Tests,baseline,Male,csf,0.0,Japan,Number,MCI due to AD
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,PET Tests,baseline,Male,pet,0.0,Japan,Number,MCI due to AD
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2025,169,CSF Tests,bbbm_testing,Female,csf,0.0,Japan,Number,MCI due to AD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139995,95_plus,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2100,446,Averted PET Tests,bbbm_testing,Male,pet,12.0,Brazil,Number,MCI due to AD
1139996,95_plus,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2100,446,Averted CSF Tests,bbbm_testing_and_treatment,Female,csf,28.0,Brazil,Number,MCI due to AD
1139997,95_plus,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2100,446,Averted PET Tests,bbbm_testing_and_treatment,Female,pet,35.0,Brazil,Number,MCI due to AD
1139998,95_plus,/mnt/team/simulation_science/pub/models/vivari...,baseline_testing,testing,2100,446,Averted CSF Tests,bbbm_testing_and_treatment,Male,csf,8.0,Brazil,Number,MCI due to AD


In [29]:
csf_pet_tests_output = results.summarize_and_beautify(csf_pet_tests_prepped, 'disease_stage')
csf_pet_tests_output

Timer SummarizingTimer started
2025-11-20 23:07:00.222434
2025-11-20 23:07:05.667660
2025-11-20 23:16:00.125153
Elapsed time: 539.9176 seconds


,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,25_to_29,Female,MCI due to AD,Reference,CSF Tests,Number,0.000000,0.000000,0.000000
1,2025,Brazil,25_to_29,Male,MCI due to AD,Reference,CSF Tests,Number,0.000000,0.000000,0.000000
2,2025,Brazil,25_to_29,Female,MCI due to AD,BBBM Testing Only,CSF Tests,Number,0.000000,0.000000,0.000000
3,2025,Brazil,25_to_29,Male,MCI due to AD,BBBM Testing Only,CSF Tests,Number,0.000000,0.000000,0.000000
4,2025,Brazil,25_to_29,Female,MCI due to AD,BBBM Testing and Treatment,CSF Tests,Number,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
455995,2100,United States of America,95_plus,Male,MCI due to AD,BBBM Testing and Treatment,Averted PET Tests,Number,27.118557,10.574914,52.686141
455996,2100,United States of America,95_plus,Female,MCI due to AD,BBBM Testing Only,Averted PET Tests,Rate,0.000072,0.000039,0.000115
455997,2100,United States of America,95_plus,Male,MCI due to AD,BBBM Testing Only,Averted PET Tests,Rate,0.000043,0.000019,0.000077
455998,2100,United States of America,95_plus,Female,MCI due to AD,BBBM Testing and Treatment,Averted PET Tests,Rate,0.000072,0.000039,0.000115


In [30]:
# Save CSF/PET testing results
csf_pet_tests_output.to_csv(output_dir / 'csf_pet_tests.csv', index=False)

# Treatment Counts

In [31]:
treatments_filters = [
    ('sub_entity', 'in',
     ['susceptible_to_treatment_to_waiting_for_treatment',
      'waiting_for_treatment_to_full_effect_long',
      'waiting_for_treatment_to_full_effect_short']
    )
]

with Timer():
    treatments = results.load_measure_from_batch_runs(
        'transition_count_treatment', False, filters=treatments_filters
    )
print(len(treatments), 'rows')
treatments.head()

Timer BatchLoadingTimer started
[('event_year', '>=', '2025'), ('sub_entity', 'in', ['susceptible_to_treatment_to_waiting_for_treatment', 'waiting_for_treatment_to_full_effect_long', 'waiting_for_treatment_to_full_effect_short'])]
{'all': PosixPath('/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/results/model8.7/model_spec/2025_11_05_15_36_29/results')}
{'United States of America': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/united_states_of_america.hdf', 'Brazil': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/brazil.hdf', 'China': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/china.hdf', 'Germany': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/germany.hdf', 'Israel': '/mnt/team/simulation_science/pub/models/vivarium_csu_alzheimers/artifacts/model8.3/israel.hdf', 'Japan': '/mnt/team/simulation_science/pub/models/vivar

,age_group,artifact_path,entity,entity_type,event_year,input_draw,measure,scenario,sex,sub_entity,value,location
0,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Female,waiting_for_treatment_to_full_effect_long,0.0,Japan
1,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Female,waiting_for_treatment_to_full_effect_short,0.0,Japan
2,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Female,susceptible_to_treatment_to_waiting_for_treatment,0.0,Japan
3,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Male,waiting_for_treatment_to_full_effect_long,0.0,Japan
4,25_to_29,/mnt/team/simulation_science/pub/models/vivari...,treatment,cause,2025,169,transition_count,baseline,Male,waiting_for_treatment_to_full_effect_short,0.0,Japan


In [32]:
start_treatment = [
        'waiting_for_treatment_to_full_effect_long',
        'waiting_for_treatment_to_full_effect_short']
treatments.query("sub_entity in @start_treatment and value>0").age_group.unique()

['60_to_64', '65_to_69', '70_to_74', '75_to_79', '80_to_84']
Categories (15, object): ['25_to_29' < '30_to_34' < '35_to_39' < '40_to_44' ... '80_to_84' < '85_to_89' < '90_to_94' < '95_plus']

In [33]:
treatments.query("sub_entity=='susceptible_to_treatment_to_waiting_for_treatment' and value>0").age_group.unique()

['60_to_64', '65_to_69', '70_to_74', '75_to_79']
Categories (15, object): ['25_to_29' < '30_to_34' < '35_to_39' < '40_to_44' ... '80_to_84' < '85_to_89' < '90_to_94' < '95_plus']

In [34]:
treatments_prepped = results.process_treatments(treatments, mslt_results_prepped)
treatments_prepped

,age_group,event_year,input_draw,measure,scenario,sex,value,location,disease_stage
0,60_to_64,2025,169,Medication Completion,baseline,Female,0.000000,Japan,Preclinical AD
1,60_to_64,2025,169,Medication Discontinuation,baseline,Female,0.000000,Japan,Preclinical AD
2,60_to_64,2025,169,Medication Completion,baseline,Male,0.000000,Japan,Preclinical AD
3,60_to_64,2025,169,Medication Discontinuation,baseline,Male,0.000000,Japan,Preclinical AD
4,60_to_64,2025,169,Medication Completion,bbbm_testing,Female,0.000000,Japan,Preclinical AD
...,...,...,...,...,...,...,...,...,...
516795,75_to_79,2100,169,Medication Initiation,BBBM Testing and Treatment,Male,42594.508978,United States of America,Susceptible
516796,75_to_79,2100,258,Medication Initiation,BBBM Testing and Treatment,Male,40580.387735,United States of America,Susceptible
516797,75_to_79,2100,323,Medication Initiation,BBBM Testing and Treatment,Male,38815.152311,United States of America,Susceptible
516798,75_to_79,2100,446,Medication Initiation,BBBM Testing and Treatment,Male,40402.097304,United States of America,Susceptible


In [35]:
with Timer():
    treatments_output = results.summarize_and_beautify(treatments_prepped, 'disease_stage')

print(len(treatments_output), 'rows')
treatments_output.head()

Timer SummarizingTimer started
2025-11-20 23:16:40.202183
2025-11-20 23:16:42.360230
2025-11-20 23:19:51.266045
Elapsed time: 191.0721 seconds
Elapsed time: 0:03:11.072364
161120 rows


,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,60_to_64,Female,Preclinical AD,Reference,Medication Completion,Number,0.0,0.0,0.0
1,2025,Brazil,60_to_64,Male,Preclinical AD,Reference,Medication Completion,Number,0.0,0.0,0.0
2,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing Only,Medication Completion,Number,0.0,0.0,0.0
3,2025,Brazil,60_to_64,Male,Preclinical AD,BBBM Testing Only,Medication Completion,Number,0.0,0.0,0.0
4,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing and Treatment,Medication Completion,Number,0.0,0.0,0.0


In [36]:
# Save treatments output
treatments_output.to_csv(output_dir / 'medication.csv', index=False)

# Check some stuff

In [37]:
treatments_output

,Year,Location,Age,Sex,Disease Stage,Scenario,Measure,Metric,Mean,95% UI Lower,95% UI Upper
0,2025,Brazil,60_to_64,Female,Preclinical AD,Reference,Medication Completion,Number,0.000000,0.000000,0.000000
1,2025,Brazil,60_to_64,Male,Preclinical AD,Reference,Medication Completion,Number,0.000000,0.000000,0.000000
2,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing Only,Medication Completion,Number,0.000000,0.000000,0.000000
3,2025,Brazil,60_to_64,Male,Preclinical AD,BBBM Testing Only,Medication Completion,Number,0.000000,0.000000,0.000000
4,2025,Brazil,60_to_64,Female,Preclinical AD,BBBM Testing and Treatment,Medication Completion,Number,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
161115,2100,United States of America,80_to_84,Male,Preclinical AD,Reference,Medication Initiation,Rate,0.000000,0.000000,0.000000
161116,2100,United States of America,80_to_84,Female,Preclinical AD,BBBM Testing Only,Medication Initiation,Rate,0.000000,0.000000,0.000000
161117,2100,United States of America,80_to_84,Male,Preclinical AD,BBBM Testing Only,Medication Initiation,Rate,0.000000,0.000000,0.000000
161118,2100,United States of America,80_to_84,Female,Preclinical AD,BBBM Testing and Treatment,Medication Initiation,Rate,0.000132,0.000019,0.000355


In [38]:
treatments_output.query("`Disease Stage` == 'Preclinical AD'").Year.unique()

[2025, 2026, 2027, 2028, 2029, ..., 2096, 2097, 2098, 2099, 2100]
Length: 76
Categories (76, int64): [2025, 2026, 2027, 2028, ..., 2097, 2098, 2099, 2100]

In [39]:
treatments_prepped.age_group.unique()

['60_to_64', '65_to_69', '70_to_74', '75_to_79', '80_to_84']
Categories (5, object): ['60_to_64', '65_to_69', '70_to_74', '75_to_79', '80_to_84']

In [40]:
treatment_rates = results.calculate_rate(treatments_prepped)
treatment_rates

,age_group,event_year,input_draw,location,sex,disease_stage,measure,scenario,value,metric
0,60_to_64,2025,169,Brazil,Female,Preclinical AD,Medication Completion,baseline,0.000000,Rate
1,60_to_64,2025,169,Brazil,Female,Preclinical AD,Medication Discontinuation,baseline,0.000000,Rate
2,60_to_64,2025,169,Brazil,Female,Preclinical AD,Medication Completion,bbbm_testing,0.000000,Rate
3,60_to_64,2025,169,Brazil,Female,Preclinical AD,Medication Discontinuation,bbbm_testing,0.000000,Rate
4,60_to_64,2025,169,Brazil,Female,Preclinical AD,Medication Completion,bbbm_testing_and_treatment,0.000000,Rate
...,...,...,...,...,...,...,...,...,...,...
516795,80_to_84,2100,457,United States of America,Male,Preclinical AD,Medication Initiation,baseline,0.000000,Rate
516796,80_to_84,2100,457,United States of America,Male,Preclinical AD,Medication Initiation,bbbm_testing,0.000000,Rate
516797,80_to_84,2100,457,United States of America,Male,Preclinical AD,Medication Initiation,bbbm_testing,0.000000,Rate
516798,80_to_84,2100,457,United States of America,Male,Preclinical AD,Medication Initiation,bbbm_testing_and_treatment,0.000090,Rate


In [41]:
treatment_rates.query("disease_stage=='Susceptible'")

,age_group,event_year,input_draw,location,sex,disease_stage,measure,scenario,value,metric
12,60_to_64,2025,169,Brazil,Female,Susceptible,Medication Initiation,BBBM Testing Only,0.000000,Rate
13,60_to_64,2025,169,Brazil,Female,Susceptible,Medication Initiation,BBBM Testing and Treatment,0.000000,Rate
26,60_to_64,2025,169,Brazil,Male,Susceptible,Medication Initiation,BBBM Testing Only,0.000000,Rate
27,60_to_64,2025,169,Brazil,Male,Susceptible,Medication Initiation,BBBM Testing and Treatment,0.000000,Rate
40,60_to_64,2025,169,China,Female,Susceptible,Medication Initiation,BBBM Testing Only,0.000000,Rate
...,...,...,...,...,...,...,...,...,...,...
425571,75_to_79,2100,457,United Kingdom,Male,Susceptible,Medication Initiation,BBBM Testing and Treatment,0.012777,Rate
425584,75_to_79,2100,457,United States of America,Female,Susceptible,Medication Initiation,BBBM Testing Only,0.000000,Rate
425585,75_to_79,2100,457,United States of America,Female,Susceptible,Medication Initiation,BBBM Testing and Treatment,0.005719,Rate
425598,75_to_79,2100,457,United States of America,Male,Susceptible,Medication Initiation,BBBM Testing Only,0.000000,Rate


In [42]:
mslt_results_prepped.query("measure.str.contains('Medication')")

,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Medication Initiation,Number,169,0.000000
8,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Medication Initiation,Number,258,0.000000
13,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Medication Initiation,Number,323,0.000000
18,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Medication Initiation,Number,446,0.000000
19,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Medication Initiation,Number,457,0.000000
...,...,...,...,...,...,...,...,...,...,...
151978,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,169,42594.508978
151983,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,258,40580.387735
151988,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,323,38815.152311
151993,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,446,40402.097304


In [43]:
mslt_results_prepped.measure.unique()

['BBBM Tests', 'BBBM Positive Tests', 'Medication Initiation']
Categories (3, object): ['BBBM Positive Tests', 'BBBM Tests', 'Medication Initiation']

# Scratchwork

In [44]:
bbbm_tests_prepped

,age_group,event_year,input_draw,location,measure,scenario,sex,value,disease_stage,metric
0,60_to_64,2025,169,Japan,BBBM Tests,baseline,Female,0.000000,Preclinical AD,Number
1,60_to_64,2025,169,Japan,BBBM Tests,baseline,Male,0.000000,Preclinical AD,Number
2,60_to_64,2025,169,Japan,BBBM Tests,bbbm_testing,Female,0.000000,Preclinical AD,Number
3,60_to_64,2025,169,Japan,BBBM Tests,bbbm_testing,Male,0.000000,Preclinical AD,Number
4,60_to_64,2025,169,Japan,BBBM Tests,bbbm_testing_and_treatment,Female,0.000000,Preclinical AD,Number
...,...,...,...,...,...,...,...,...,...,...
303995,75_to_79,2100,169,United States of America,BBBM Positive Tests,BBBM Testing and Treatment,Male,141981.696594,Susceptible,Number
303996,75_to_79,2100,258,United States of America,BBBM Positive Tests,BBBM Testing and Treatment,Male,135267.959115,Susceptible,Number
303997,75_to_79,2100,323,United States of America,BBBM Positive Tests,BBBM Testing and Treatment,Male,129383.841038,Susceptible,Number
303998,75_to_79,2100,446,United States of America,BBBM Positive Tests,BBBM Testing and Treatment,Male,134673.657679,Susceptible,Number


In [45]:
column_name_map = {
            'event_year': 'Year',
            'age_group': 'Age',
            'location': 'Location',
            'sex': 'Sex',
            'scenario': 'Scenario',
            'measure': 'Measure',
            'metric': 'Metric',
            'disease_stage': 'Disease Stage',
            'input_draw': 'Draw',
            'value': 'Value',
            'mean': 'Mean',
            'lower': '95% UI Lower',
            'upper': '95% UI Upper',
        }
reverse_map = {v: k for k, v in column_name_map.items()}
reverse_map

{'Year': 'event_year',
 'Age': 'age_group',
 'Location': 'location',
 'Sex': 'sex',
 'Scenario': 'scenario',
 'Measure': 'measure',
 'Metric': 'metric',
 'Disease Stage': 'disease_stage',
 'Draw': 'input_draw',
 'Value': 'value',
 'Mean': 'mean',
 '95% UI Lower': 'lower',
 '95% UI Upper': 'upper'}

In [46]:
temp = mslt_results.rename(columns=reverse_map)
temp

,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
0,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,113,0.000000
1,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,13,0.000000
2,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,158,0.000000
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,169,0.000000
4,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,177,0.000000
...,...,...,...,...,...,...,...,...,...,...
151995,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,46,41101.303607
151996,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,460,41691.942867
151997,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,480,39195.330257
151998,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Improper Medication Uses,Number,499,40424.816911


In [47]:
pd.concat([bbbm_tests_prepped, temp])

,age_group,event_year,input_draw,location,measure,scenario,sex,value,disease_stage,metric
0,60_to_64,2025,169,Japan,BBBM Tests,baseline,Female,0.000000,Preclinical AD,Number
1,60_to_64,2025,169,Japan,BBBM Tests,baseline,Male,0.000000,Preclinical AD,Number
2,60_to_64,2025,169,Japan,BBBM Tests,bbbm_testing,Female,0.000000,Preclinical AD,Number
3,60_to_64,2025,169,Japan,BBBM Tests,bbbm_testing,Male,0.000000,Preclinical AD,Number
4,60_to_64,2025,169,Japan,BBBM Tests,bbbm_testing_and_treatment,Female,0.000000,Preclinical AD,Number
...,...,...,...,...,...,...,...,...,...,...
151995,75_to_79,2100,46,United States of America,Improper Medication Uses,BBBM Testing and Treatment,Male,41101.303607,Susceptible,Number
151996,75_to_79,2100,460,United States of America,Improper Medication Uses,BBBM Testing and Treatment,Male,41691.942867,Susceptible,Number
151997,75_to_79,2100,480,United States of America,Improper Medication Uses,BBBM Testing and Treatment,Male,39195.330257,Susceptible,Number
151998,75_to_79,2100,499,United States of America,Improper Medication Uses,BBBM Testing and Treatment,Male,40424.816911,Susceptible,Number


In [48]:
temp.query("measure.str.contains('Medication') and value > 0 and scenario=='BBBM Testing Only'")

,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
1000,2030,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,113,24115.316094
1001,2030,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,13,25432.967559
1002,2030,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,158,24846.791963
1003,2030,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,169,24018.184411
1004,2030,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,177,21771.561248
...,...,...,...,...,...,...,...,...,...,...
151995,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,46,41101.303607
151996,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,460,41691.942867
151997,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,480,39195.330257
151998,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing Only,Improper Medication Uses,Number,499,40424.816911


In [49]:
temp.measure.unique()

array(['BBBM Tests', 'BBBM False Positive Tests',
       'Improper Medication Uses'], dtype=object)

In [50]:
temp.replace(
    {'measure': {'BBBM False Positive Tests': 'BBBM Positive Tests',
 'Improper Medication Uses': 'Medication Initiation'}})


,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
0,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,113,0.000000
1,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,13,0.000000
2,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,158,0.000000
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,169,0.000000
4,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,177,0.000000
...,...,...,...,...,...,...,...,...,...,...
151995,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,46,41101.303607
151996,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,460,41691.942867
151997,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,480,39195.330257
151998,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,499,40424.816911


In [51]:
bbbm_tests_prepped.dtypes

age_group        category
event_year       category
input_draw       category
location         category
measure          category
scenario         category
sex              category
value             float64
disease_stage    category
metric           category
dtype: object

In [52]:
testing = temp.query("scenario=='BBBM Testing Only'").drop(columns='scenario')
treatment = temp.query("scenario=='BBBM Testing and Treatment'").drop(columns='scenario')
# This shows that the whole dataframe was copied to the testing-only
# scenario, including medication initiations, which is incorrect
results.ops.compare_values(testing, treatment)

age_group,disease_stage,event_year,input_draw,location,measure,metric,sex


In [53]:
mslt_results_processed = results.process_mslt_results(mslt_results)
mslt_results_processed

,event_year,location,age_group,sex,disease_stage,scenario,measure,metric,input_draw,value
3,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,169,0.000000
8,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,258,0.000000
13,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,323,0.000000
18,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,446,0.000000
19,2025,Brazil,60_to_64,Female,Susceptible,BBBM Testing Only,BBBM Tests,Number,457,0.000000
...,...,...,...,...,...,...,...,...,...,...
151978,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,169,42594.508978
151983,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,258,40580.387735
151988,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,323,38815.152311
151993,2100,United States of America,75_to_79,Male,Susceptible,BBBM Testing and Treatment,Medication Initiation,Number,446,40402.097304


In [54]:
mslt_results_processed.input_draw.unique()

[169, 258, 323, 446, 457]
Categories (5, int64): [169, 258, 323, 446, 457]

In [55]:
print_memory_usage(mslt_results_processed)

4.565503 MB 


In [56]:
mslt_results_processed.measure.unique()

['BBBM Tests', 'BBBM Positive Tests', 'Medication Initiation']
Categories (3, object): ['BBBM Positive Tests', 'BBBM Tests', 'Medication Initiation']

In [57]:
!date

Thu Nov 20 23:19:59 PST 2025
